In [6]:
from generating_samples import *
from fitting_models import *
from evaluation_metrics import *

In [52]:
sqlite_file_name = "samples.sqlite"

SGW = SampleParamsWrapper(nr_variables=5,
                          nr_samples=50,
                          random_seed=0,
                          sample_init=np.zeros((5, )))

SGW.graph_generator = HubGraphGenerator(nr_hubs=1)
SGW.weight_assigner = Bimodal_Distr_Weight_Assigner(neg_mean=-1, pos_mean=0.5, 
                                                    threshold=1, std=0)
SGW.model = TPGM(R=10)
SGW.sampler = TPGMGibbsSampler(burn_in=100, thinning_nr=100)

SGW.generate_samples_and_save("TestTPGM", sqlite_file_name)
SPS = SampleParamsWrapper.load_samples("TestTPGM", sqlite_file_name)

100%|██████████| 5001/5001 [00:01<00:00, 2512.17it/s]


In [65]:
samples_file_name = "samples.sqlite"
samples_id = "TestTPGM"

fit_id = "debugEverything"
fit_file_name = "fit_models.sqlite"

FPW = FitParamsWrapper(random_seed=2,
                       samples_file_name=samples_file_name,
                       samples_id=samples_id)

FPW.model = TPGM(R=10)
FPW.fitter = Prox_Grad_Fitter(alpha=0.5, early_stop_criterion='likelihood')

theta_final = FPW.fit_model_and_save(fit_id=fit_id,
                                     fit_file_name=fit_file_name,
                                     parallelize=True)

print(theta_final)


Parameters for node 0 converged in 1 iterations.

Parameters for node 1 converged in 1 iterations.

Parameters for node 3 converged in 1 iterations.

Parameters for node 4 converged in 1 iterations.

Parameters for node 2 converged in 5 iterations.
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [-0.14188743 -0.21012109 -0.0549458  -0.11046193 -0.06427448]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]


In [58]:
theta_init = SGW.model.theta
print(theta_init)

[[ 0.  0. -1.  0.  0.]
 [ 0.  0. -1.  0.  0.]
 [-1. -1.  0. -1. -1.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0. -1.  0.  0.]]


In [71]:
ww_min = EvalMetrics.SymmModes.WW_MIN
ww_max = EvalMetrics.SymmModes.WW_MAX

print(EvalMetrics.calculate_tpr_fpr_acc_nonzero(theta_init, theta_final, ww_max))
print(EvalMetrics.calculate_signed_recall(theta_init, theta_final, ww_max))
print(EvalMetrics.calculate_percentage_symmetric_signs(theta_final))
print(EvalMetrics.calculate_percentage_symmetric_values(theta_final))
print(EvalMetrics.calculate_percentage_symmetric_nonzero(theta_final))
print(EvalMetrics.calculate_percentage_sparsity(theta_init))

(1.0, 0.0, 1.0)
0.0
0.6
0.6
0.6
0.6


In [70]:
print(EvalMetrics.copy_and_symmetrize_matrix(theta_final, ww_max))

[[ 0.          0.         -0.14188743  0.          0.        ]
 [ 0.          0.         -0.21012109  0.          0.        ]
 [-0.14188743 -0.21012109 -0.0549458  -0.11046193 -0.06427448]
 [ 0.          0.         -0.11046193  0.          0.        ]
 [ 0.          0.         -0.06427448  0.          0.        ]]
